<a href="https://colab.research.google.com/github/ameyaoka/ml-notebooks/blob/main/movie_genre_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
  pip install --upgrade huggingface_hubd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement huggingface_hubd (from versions: none)
ERROR: No matching distribution found for huggingface_hubd


In [3]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.8 MB/s eta 0:00:00


In [4]:
import pandas as pd

import torch

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("datadrivenscience/movie-genre-prediction")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/54000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/36000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/datadrivenscience___parquet/datadrivenscience--movie-genre-prediction-01acd85570f2b187/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 54000
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'genre'],
        num_rows: 36000
    })
})


- Dataset consist of two sets 1. train  2. test

In [8]:
train_dataset = dataset['train']

In [9]:
train_dataset[0]

{'id': 44978,
 'movie_name': 'Super Me',
 'synopsis': 'A young scriptwriter starts bringing valuable objects back from his short nightmares of being chased by a demon. Selling them makes him rich.',
 'genre': 'fantasy'}

- movie id is not required for training .
- movie_name is our first input and synopsis is our second input
- genre is out label.


In [ ]:
dataset.set_format("pandas")
train_df = pd.DataFrame(dataset["train"][:])

In [ ]:
train_df

,id,movie_name,synopsis,genre
0,44978,Super Me,A young scriptwriter starts bringing valuable ...,fantasy
1,50185,Entity Project,A director and her friends renting a haunted h...,horror
2,34131,Behavioral Family Therapy for Serious Psychiat...,This is an educational video for families and ...,family
3,78522,Blood Glacier,Scientists working in the Austrian Alps discov...,scifi
4,2206,Apat na anino,Buy Day - Four Men Widely - Apart in Life - By...,action
...,...,...,...,...
53995,80325,Untitled Monkeyshine/Ugly Brother Project,Two loser pest control buddies find themselves...,scifi
53996,84712,The Great Swindle,A seductive woman gets involved in relationshi...,thriller
53997,69801,When Men Get Pregnant,"Duyen, a wedding dress staff, who decides to c...",romance
53998,7678,Suttu Pidikka Utharavu,The people of a crowded colony in Coimbatore c...,action


In [ ]:
train_df["synopsis"]

0        A young scriptwriter starts bringing valuable ...
1        A director and her friends renting a haunted h...
2        This is an educational video for families and ...
3        Scientists working in the Austrian Alps discov...
4        Buy Day - Four Men Widely - Apart in Life - By...
                               ...                        
53995    Two loser pest control buddies find themselves...
53996    A seductive woman gets involved in relationshi...
53997    Duyen, a wedding dress staff, who decides to c...
53998    The people of a crowded colony in Coimbatore c...
53999    Margo is a little mouse that lives quietly in ...
Name: synopsis, Length: 54000, dtype: object

In [ ]:
unique_labels =train_df['genre'].unique()

In [ ]:
unique_labels

array(['fantasy', 'horror', 'family', 'scifi', 'action', 'crime',
       'adventure', 'mystery', 'romance', 'thriller'], dtype=object)

In [ ]:
train_df.isnull().sum()

id            0
movie_name    0
synopsis      0
genre         0
dtype: int64

In [ ]:
# Let's check the distribution of different labels
import matplotlib.pyplot as plt

In [ ]:
train_df["genre"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Labels")
plt.show()

NameError: ignored

- The dataset is balanced

## tokenisation


1.     Find a list of all unique words in the dataset.

2.    Associate each word/token with a unique number. This is called vocab.

3.  Encode the dataset using the mapping present in the vocab.


- Model max length defines the maximum number of tokens that a single data sample can have, i.e. in the above case, our model DistilBert can accept text sequences of up to 512 tokens long.

- Model input names are the fields that the model will take as inputs for training and inference purposes.

In [10]:
train_dataset['synopsis']

['A young scriptwriter starts bringing valuable objects back from his short nightmares of being chased by a demon. Selling them makes him rich.',
 'A director and her friends renting a haunted house to capture paranormal events in order to prove it and become popular.',
 'This is an educational video for families and family therapists that describes the Behavioral Family Therapy approach to dealing with serious psychiatric illnesses.',
 'Scientists working in the Austrian Alps discover that a glacier is leaking a liquid that appears to be affecting local wildlife.',
 'Buy Day - Four Men Widely - Apart in Life - By Night Shadows United in One Fight Venting the Fire of their Fury Against the Hated Oppressors.',
 "On an island, things start going crazy as the islands house hold appliances go rogue. It's up to Dr. Gabrielle Martin to discover what's causing this evil, and much more.",
 'A video voyeur stalks women in the city with a digital camera until he crosses paths with beautiful mode

In [11]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [12]:
train_dataset['synopsis'][0]

'A young scriptwriter starts bringing valuable objects back from his short nightmares of being chased by a demon. Selling them makes him rich.'

In [13]:
train_dataset['movie_name'][0]

'Super Me'

In [14]:
inputs = tokenizer(train_dataset['synopsis'][0],train_dataset['movie_name'][0])

In [15]:
inputs

{'input_ids': [101, 1037, 2402, 5896, 15994, 4627, 5026, 7070, 5200, 2067, 2013, 2010, 2460, 15446, 1997, 2108, 13303, 2011, 1037, 5698, 1012, 4855, 2068, 3084, 2032, 4138, 1012, 102, 3565, 2033, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

- token_type_id tells model which part of input is first sentence and which part is second sentence.

In [31]:
def tokenize_function(example):
    return tokenizer(example["movie_name"], example["synopsis"], truncation=True, padding =True ,return_tensors="pt")

In [32]:

tokenized_train = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/54000 [00:00<?, ? examples/s]

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

- **Our tokenized dataset consist of id , movie_name , synopsis genre input_id token_type_id , attnetion mask .**
- **we need attetion_mask , inputids , labels , tokent_type_ids**


In [33]:
tokenized_train['train'][0]

{'id': 44978,
 'movie_name': 'Super Me',
 'synopsis': 'A young scriptwriter starts bringing valuable objects back from his short nightmares of being chased by a demon. Selling them makes him rich.',
 'genre': 'fantasy',
 'input_ids': [101,
  3565,
  2033,
  102,
  1037,
  2402,
  5896,
  15994,
  4627,
  5026,
  7070,
  5200,
  2067,
  2013,
  2010,
  2460,
  15446,
  1997,
  2108,
  13303,
  2011,
  1037,
  5698,
  1012,
  4855,
  2068,
  3084,
  2032,
  4138,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [34]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [21]:

tokenized_dataset = tokenized_train.rename_column('genre',"labels")

tokenized_dataset.set_format('torch')


In [22]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 54000
    })
    test: Dataset({
        features: ['id', 'movie_name', 'synopsis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36000
    })
})

In [24]:
tokenized_dataset  =tokenized_dataset.remove_columns(['id','movie_name','synopsis'])

In [25]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 54000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36000
    })
})

In [37]:
id2label = {0:'fantasy',1:'horror',2:'family',3:'scifi',4:'action',5:'crime',6:'adventure',7:'mystery',8:'romance',9:'thriller'}
label2id = {"fantasy": 0, "horror": 1,'family':2,'scifi':3,'action':4,'crime':5,'adventure':6,'mystery':7,'romance':8,'thriller':9}

In [40]:

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset ["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=8, collate_fn=data_collator
)

In [41]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

ValueError: ignored